In [27]:
import pandas as pd

df = pd.read_csv("defi_curated.csv")
# print(df)
print(df[" stake_token"][0])
df.dtypes

nan


name                   object
 created_at            object
 deposit_signature     object
 withdraw_signature    object
 proxy_contract        object
 stake_token           object
dtype: object

In [28]:
for i in range(0, 1814, 1):
    if df[" stake_token"][i] != df[" proxy_contract"][i]:
        # print(i)
        df[" stake_token"][i] = df[" proxy_contract"][i]

print(df[" stake_token"][0])
print(df)

 0xcd18eaa163733da39c232722cbc4e8940b1d8888
                        name  created_at  deposit_signature  \
0     address.dapp.sablier_0        None               None   
1       address.dapp.curve_0        None               None   
2       address.dapp.curve_1        None               None   
3       address.dapp.curve_2        None               None   
4       address.dapp.curve_3        None               None   
...                      ...         ...                ...   
1810    polygon-pos-bridge_3        None               None   
1811    polygon-pos-bridge_4        None               None   
1812              arbitrum_0        None               None   
1813              arbitrum_1        None               None   
1814          bitkeep-swap_0        None               None   

      withdraw_signature                               proxy_contract  \
0                   None   0xcd18eaa163733da39c232722cbc4e8940b1d8888   
1                   None   0xa2b47e3d5c44877cca798226

In [30]:
df.to_csv("defi.csv", index=None)

In [1]:
import pandas as pd
import csv
from time import sleep
import requests as rq
import json
import os
import ast

folder_path = "logicContract2"
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]

df_list = []  # 存放每个CSV文件的DataFrame

for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

# 将所有DataFrame合并为一个
combined_df = pd.concat(df_list, ignore_index=True)

data = pd.DataFrame(combined_df)
# data.to_csv("1111.csv")

In [2]:
data["logic"] = data["logic"].apply(lambda x: ast.literal_eval(x))
for dapp, proxy, logic in zip(data["dapp"], data["proxy"], data["logic"]):
    for address in logic:
        print(address)

0x48f9505ae0c26bffbf4a0fd5054296f4eeb28c34
0x611c49e049667f67f35e1e271b89299ce00e513a
0xb6bf84cbe86d786a3a66fd1a1fa77d1e521596ea
0x5966e2a40e65954d229f020e989be296ec116746
0xab98c77a164c94d7dfb32d2766f6d8ccb84ab344
0x7f59672c6d909d97013853ae936ed1e3652c6024
0xdd3b36b5d8d63da10dea1dee649af86b085bd3bb
0x1f9b696272130de4c3b038376ae8afbdd14580e6
0x55f71e9de1f2b0ddd2bf58cab76c0595c0972418
0x7388938fe677a9c9563edba07213fd4a05b930be
0x9a60bcbe43b37e076c16889bfdc82dd13b014b31
0xce101ac2373815b171b7c7c38d698261f20f75d6
0x864c54e97be6f3855e5fc9cda573e8e9a79d2158
0x3e847121dbc33ca09aa5c75a5208701764af8389
0xa430b280b61f3314ec900690abc229ce2b626e5c
0xd978eaef5879fa7b3a7fad7edeabb34f11277fe3
0x8f10db1e9f15837d8153f95d358e8cfed402f7b3
0x5318731e1fb85d3743f9f8412d9441459d3d9f00
0x31db4db73164188a6a9dec4d3023c73e4dfb7e5f
0x9a882177a99373cc42cdbcdd8d0f4efcb038f280
0x4b6d92828d9014246d47bc2db39a5ea88762e016
0x5678864d7f05799f45982d542ea4f676dac7ef17
0xefa9d1e4d826fdc13edd2d166b7a2eb1ae73dd12
0x22bcbd242

In [3]:
def make_dir(path):
    folders = []
    while not os.path.isdir(path):
        path, suffix = os.path.split(path)
        folders.append(suffix)
    for folder in folders[::-1]:
        path = os.path.join(path, folder)
        os.mkdir(path)

def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except ValueError as e:
        return False
    return True

send_headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    "Connection": "keep-alive",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9"}

contract_data = []

def download_url(url, retry=5, retry_delay=5, throw_if_fail=False):
    resp = rq.get(url, headers=send_headers, allow_redirects=False)

    if resp.status_code != 200:
        if retry > 0:
            sleep(retry_delay)
            f'Download source failed for {url}, status {resp.status_code}, retry in {retry_delay} secs'
            return download_url(url, retry-1, retry_delay)
        else:
            if throw_if_fail:
                raise Exception(f'Download source abort for {url}, status {resp.status_code}')
            return
    return resp

for contract_name, proxy, logic in zip(combined_df["dapp"], combined_df["proxy"], combined_df["logic"]):
    for contract_address in logic:
        make_dir("SourceAndABI2/" + contract_name + "/" + proxy[:8] + "/" + contract_address[:8])
        root = "./SourceAndABI2/"+ contract_name + "/" + proxy[:8] + "/" + contract_address[:8] + "/"

        abi_link = 'http://api.etherscan.io/api?module=contract&action=getabi&address=' + \
                        contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
        abi_output = download_url(abi_link)
        abi_json_res = abi_output.json()
        abi_json = abi_json_res["result"]
        abi_file = open(root+"abi.json", 'w+', encoding='UTF-8')
        abi_file.write(abi_json)
        # with open(root+contract_address+"/"+"abi.json", 'w+') as f:
        #     json.dump(abi_json, f)

        curl_link = 'http://api.etherscan.io/api?module=contract&action=getsourcecode&address=' + \
                        contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
        print(curl_link)
        output = download_url(curl_link)
        json_res = output.json()

        # json_data = json_res["result"]
        # print(json_data)
        json_file = open(root+"outputJson.json", 'w+', encoding='UTF-8')
        # json_file.write(json_data)
        with open(root+"outputJson.json", 'w+') as f:
            json.dump(json_res, f)

        if "result" in json_res:
            source_code = json_res["result"][0]["SourceCode"]
            make_dir(root+"SourceCode")
            if is_json(source_code):
                res = json.loads(source_code)
                for key in res:
                    sol_file = open(root+"SourceCode"+"/"+key, 'w+', encoding='UTF-8')
                    sol_file.write(res[key]["content"])
                    if "contract "+contract_name in res[key]["content"]:
                        contract_data_element = [contract_address, contract_name, key]
                        # contract_data.append(contract_data_element)
                        df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                        df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
                    sol_file.close()
            elif len(source_code) > 2 and source_code[0] == source_code[1] == "{":
                new_code = source_code[1:-1]
                res = json.loads(new_code)
                sources = res["sources"]
                for name in sources:
                    _dir, _file = os.path.split(root+"SourceCode"+name)
                    # print(name)
                    make_dir(_dir)
                    # print(_dir)
                    sol_file = open(root+"SourceCode"+name, 'w+', encoding='UTF-8')
                    sol_file.write(sources[name]["content"])
                    
                    if "contract "+contract_name in sources[name]["content"]:
                        contract_data_element = [contract_address, contract_name, name]
                        # contract_data.append(contract_data_element)
                        df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                        df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
                    sol_file.close()
            else:
                sol_file = open(root+"SourceCode" + "/" +
                            contract_name + ".sol", 'w+', encoding='UTF-8')
                sol_file.write(source_code)
                contract_data_element = [contract_address, contract_name, contract_name]
                df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
                sol_file.close()

http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x48f9505ae0c26bffbf4a0fd5054296f4eeb28c34&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x611c49e049667f67f35e1e271b89299ce00e513a&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xb6bf84cbe86d786a3a66fd1a1fa77d1e521596ea&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x5966e2a40e65954d229f020e989be296ec116746&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0xab98c77a164c94d7dfb32d2766f6d8ccb84ab344&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=getsourcecode&address=0x7f59672c6d909d97013853ae936ed1e3652c6024&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW
http://api.etherscan.io/api?module=contract&action=g

In [2]:
# 单个合约调试

import pandas as pd
import csv
from time import sleep
import requests as rq
import json
import os
import ast

def make_dir(path):
    folders = []
    while not os.path.isdir(path):
        path, suffix = os.path.split(path)
        folders.append(suffix)
    for folder in folders[::-1]:
        path = os.path.join(path, folder)
        os.mkdir(path)

def is_json(myjson):
    try:
        json_object = json.loads(myjson)
    except ValueError as e:
        return False
    return True

send_headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    "Connection": "keep-alive",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "zh-CN,zh;q=0.9"}

contract_data = []

def download_url(url, retry=5, retry_delay=5, throw_if_fail=False):
    resp = rq.get(url, headers=send_headers, allow_redirects=False)

    if resp.status_code != 200:
        if retry > 0:
            sleep(retry_delay)
            f'Download source failed for {url}, status {resp.status_code}, retry in {retry_delay} secs'
            return download_url(url, retry-1, retry_delay)
        else:
            if throw_if_fail:
                raise Exception(f'Download source abort for {url}, status {resp.status_code}')
            return
    return resp

contract_address = "0xe7f445b93eb9cdabfe76541cc43ff8de930a58e6"
contract_address = contract_address.lower()
make_dir(f"./singleTest/{contract_address}")
root = f"./singleTest/{contract_address}/"

abi_link = 'http://api.etherscan.io/api?module=contract&action=getabi&address=' + \
                contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
abi_output = download_url(abi_link)
abi_json_res = abi_output.json()
print(abi_json_res)
abi_json = abi_json_res["result"]
abi_file = open(root+"abi.json", 'w+', encoding='UTF-8')
abi_file.write(abi_json)
# with open(root+contract_address+"/"+"abi.json", 'w+') as f:
#     json.dump(abi_json, f)

curl_link = 'http://api.etherscan.io/api?module=contract&action=getsourcecode&address=' + \
                contract_address+'&apikey=BEYXCX8H6CDFDE5MUU4WQ6NUTIUKM1ZVQW'
print(curl_link)
output = download_url(curl_link)
json_res = output.json()

# json_data = json_res["result"]
# print(json_data)
json_file = open(root+"outputJson.json", 'w+', encoding='UTF-8')
# json_file.write(json_data)
with open(root+"outputJson.json", 'w+') as f:
    json.dump(json_res, f)

if "result" in json_res:
    source_code = json_res["result"][0]["SourceCode"]
    contract_name = json_res["result"][0]["ContractName"]
    make_dir(root+"SourceCode")
    if is_json(source_code):
        print("111")
        res = json.loads(source_code)
        for key in res:
            sol_file = open(root+"SourceCode"+"/"+key, 'w+', encoding='UTF-8')
            sol_file.write(res[key]["content"])
            if "contract "+contract_name in res[key]["content"]:
                contract_data_element = [contract_address, contract_name, key]
                # contract_data.append(contract_data_element)
                df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
            sol_file.close()
    elif len(source_code) > 2 and source_code[0] == source_code[1] == "{":
        new_code = source_code[1:-1]
        res = json.loads(new_code)
        sources = res["sources"]
        for name in sources:
            _dir, _file = os.path.split(root+"SourceCode"+"/"+name)
            # print("name", name)
            make_dir(_dir)
            # print("_dir", _dir)
            sol_file = open(root+"SourceCode"+"/"+name, 'w+', encoding='UTF-8')
            sol_file.write(sources[name]["content"])
            
            # if "contract "+contract_name in sources[name]["content"]:
                # contract_data_element = [contract_address, contract_name, name]
                # print(name)
                # print(_dir)
                # contract_data.append(contract_data_element)
                # df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
                # df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
            sol_file.close()
    else:
        sol_file = open(root+"SourceCode" + "/" +
                    contract_name + ".sol", 'w+', encoding='UTF-8')
        sol_file.write(source_code)
        contract_data_element = [contract_address, contract_name, contract_name]
        df = pd.DataFrame([contract_data_element],columns=['ContractAddress','ContractName','ContractPath'])
        df.to_csv('./contract.csv',sep=',',mode='a',index=False,header=None)
        sol_file.close()

{'status': '1', 'message': 'OK', 'result': '[{"inputs":[{"internalType":"address","name":"_underlying","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"beneficiary","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false